# Data cleaning and descriptive stats

Overview of today's learning goals:

  1. Introduce pandas
  2. Load data files
  3. Clean and process data
  4. Select, filter, and slice data from a dataset
  5. Descriptive stats: central tendency and dispersion
  6. Merging and concatenating datasets
  7. Grouping and summarizing data

In [ ]:
!uv pip install numpy pandas

In [ ]:
# something new: import these packages to work with data
import numpy as np
import pandas as pd
from google.colab import drive

# mount your google drive
drive.mount("/content/drive")

## 1. Introducing pandas

https://pandas.pydata.org/

In [ ]:
# review: a python list is a built-in data type
my_list = [8, 6, 4, 2]
my_list

In [ ]:
# a numpy array is like a list
# but faster, more compact, and lots more features
my_array = np.array(my_list)
my_array

pandas has two primary data structures we will work with: Series and DataFrames

### 1a. pandas Series

In [ ]:
# a pandas series is based on a numpy array: it's fast, compact, and has more functionality
# perhaps most notably, it has an index which allows you to work naturally with tabular data
my_series = pd.Series(my_list)
my_series

In [ ]:
# look at a list-representation of the index
my_series.index.tolist()

In [ ]:
# look at the series' values themselves
my_series.values

In [ ]:
# what's the data type of the series' values?
type(my_series.values)

In [ ]:
# what's the data type of the individual values themselves?
my_series.dtype

### 1b. pandas DataFrames

In [ ]:
# a dict can contain multiple lists and label them
my_dict = {
    "hh_income": [75125, 22075, 31950, 115400],
    "home_value": [525000, 275000, 395000, 985000],
}
my_dict

In [ ]:
# a pandas dataframe can contain one or more columns
# each column is a pandas series
# each row is a pandas series
# you can create a dataframe by passing in a list, array, series, or dict
df = pd.DataFrame(my_dict)
df

In [ ]:
# the row labels in the index are accessed by the .index attribute of the DataFrame object
df.index.tolist()

In [ ]:
# the column labels are accessed by the .columns attribute of the DataFrame object
df.columns

In [ ]:
# the data values are accessed by the .values attribute of the DataFrame object
# this is a numpy (two-dimensional) array
df.values

## 2. Loading data

In practice, you'll work with data by loading a dataset file into pandas. CSV is the most common format. But pandas can also ingest tab-separated data, JSON, and proprietary file formats like Excel .xlsx files, Stata, SAS, and SPSS.

Below, notice what pandas's `read_csv` function does:

1. recognize the header row and get its variable names
1. read all the rows and construct a pandas DataFrame (an assembly of pandas Series rows and columns)
1. construct a unique index, beginning with zero
1. infer the data type of each variable (ie, column)

In [ ]:
# load a data file
# note the dtype argument! always specify that fips codes are strings, otherwise pandas guesses int
filepath = "https://raw.githubusercontent.com/gboeing/ppd534/main/data/census_tracts_data_la.csv"
df = pd.read_csv(filepath, dtype={"GEOID10": str})

In [ ]:
# dataframe shape as rows, columns
df.shape

In [ ]:
# or use len to just see the number of rows
len(df)

In [ ]:
# view the dataframe's "head"
df.head()

In [ ]:
# view the dataframe's "tail"
df.tail()

#### What are these data?

I gathered them from the census bureau (2017 5-year tract-level ACS) for you, then gave them meaningful variable names. It's a set of socioeconomic variables across all LA County census tracts:

|column|description|
|------|-----------|
|total_pop|Estimate!!SEX AND AGE!!Total population|
|median_age|Estimate!!SEX AND AGE!!Total population!!Median age (years)|
|pct_hispanic|Percent Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)|
|pct_white|Percent Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!White alone|
|pct_black|Percent Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Black or African American alone|
|pct_asian|Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Asian alone|
|pct_male|Percent Estimate!!SEX AND AGE!!Total population!!Male|
|pct_single_family_home|Percent Estimate!!UNITS IN STRUCTURE!!Total housing units!!1-unit detached|
|med_home_value|Estimate!!VALUE!!Owner-occupied units!!Median (dollars)|
|med_rooms_per_home|Estimate!!ROOMS!!Total housing units!!Median rooms|
|pct_built_before_1940|Percent Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1939 or earlier|
|pct_renting|Percent Estimate!!HOUSING TENURE!!Occupied housing units!!Renter-occupied|
|rental_vacancy_rate|Estimate!!HOUSING OCCUPANCY!!Total housing units!!Rental vacancy rate|
|avg_renter_household_size|Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of renter-occupied unit|
|med_gross_rent|Estimate!!GROSS RENT!!Occupied units paying rent!!Median (dollars)|
|med_household_income|Estimate!!INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS)!!Total households!!Median household income (dollars)|
|mean_commute_time|Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Mean travel time to work (minutes)|
|pct_commute_drive_alone|Percent Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car truck or van drove alone|
|pct_below_poverty|Percent Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people|
|pct_college_grad_student|Percent Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school|
|pct_same_residence_year_ago|Percent Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Same house|
|pct_bachelors_degree|Percent Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Percent bachelor's degree or higher|
|pct_english_only|Percent Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!English only|
|pct_foreign_born|Percent Estimate!!PLACE OF BIRTH!!Total population!!Foreign born|

## 3. Clean and process data

In [ ]:
df.head(10)

In [ ]:
# data types of the columns
df.dtypes

In [ ]:
# access a single column like df['col_name']
df["med_gross_rent"].head(10)

In [ ]:
# pandas uses numpy's nan to represent null (missing) values
print(np.nan)
print(type(np.nan))

In [ ]:
# convert rent from string -> float
df["med_gross_rent"].astype(float)

Didn't work! We need to clean up the stray alphabetical characters to get a numerical value. You can do string operations on pandas Series to clean up their values

In [ ]:
# do a string replace and assign back to that column, then change type to float
df["med_gross_rent"] = df["med_gross_rent"].str.replace(" (USD)", "", regex=False)
df["med_gross_rent"] = df["med_gross_rent"].astype(float)

In [ ]:
# now clean up the income column then convert it from string -> float
# do a string replace and assign back to that column
df["med_household_income"] = df["med_household_income"].str.replace("$", "", regex=False)
df["med_household_income"] = df["med_household_income"].astype(float)

In [ ]:
# convert rent from float -> int
df["med_gross_rent"].astype(int)

You cannot store null values as type `int`, only as type `float`. You have three basic options:

  1. Keep the column as float to retain the nulls - they are often important!
  2. Drop all the rows that contain nulls if we need non-null data for our analysis
  3. Fill in all the nulls with another value if we know a reliable default value

In [ ]:
df.shape

In [ ]:
# drop rows that contain nulls
# this doesn't save result, because we didn't reassign! (in reality, want to keep the nulls here)
df.dropna(subset=["med_gross_rent"]).shape

In [ ]:
# fill in rows that contain nulls
# this doesn't save result, because we didn't reassign! (in reality, want to keep the nulls here)
df["med_gross_rent"].fillna(value=0).head(10)

In [ ]:
# more string operations: slice state fips and county fips out of the tract fips string
# assign them to new dataframe columns
df["state"] = df["GEOID10"].str.slice(0, 2)
df["county"] = df["GEOID10"].str.slice(2, 5)
df.head()

In [ ]:
# dict that maps state fips code -> state name
fips = {"04": "Arizona", "06": "California", "41": "Oregon"}

# replace fips code with state name with the replace() method
df["state"] = df["state"].replace(fips)

In [ ]:
# you can rename columns with the rename() method
# remember to reassign to save the result
df = df.rename(columns={"state": "state_name"})

In [ ]:
# you can drop columns you don't need with the drop() method
# remember to reassign to save the result
df = df.drop(columns=["county"])

In [ ]:
# inspect the cleaned-up dataframe
df.head()

In [ ]:
# save it to disk as a "clean" copy
# note the filepath
filepath = "/content/drive/My Drive/Colab Notebooks/census_tracts_data_la-clean.csv"
df.to_csv(filepath, index=False, encoding="utf-8")

In [ ]:
# and you can read it back in
pd.read_csv(filepath)

## 4. Selecting and slicing data from a DataFrame

In [ ]:
# CHEAT SHEET OF COMMON TASKS
# Operation                       Syntax           Result
# ------------------------------------------------------------
# Select column by name           df[col]          Series
# Select columns by name          df[col_list]     DataFrame
# Select row by label             df.loc[label]    Series
# Select row by integer location  df.iloc[loc]     Series
# Slice rows by label             df.loc[a:c]      DataFrame
# Select rows by boolean vector   df[mask]         DataFrame

### 4a. Select DataFrame's column(s) by name

We saw some of this a minute ago. Let's look in a bit more detail and break down what's happening.

In [ ]:
# select a single column by column name
# this is a pandas series
df["total_pop"]

In [ ]:
# select multiple columns by a list of column names
# this is a pandas dataframe that is a subset of the original
df[["total_pop", "median_age"]]

In [ ]:
# create a new column by assigning df['new_col'] to some set of values
# you can do math operations on any numeric columns
df["monthly_income"] = df["med_household_income"] / 12
df["rent_burden"] = df["med_gross_rent"] / df["monthly_income"]

# inspect the results
df[["med_household_income", "monthly_income", "med_gross_rent", "rent_burden"]].head()

### 4b. Select row(s) by label

In [ ]:
# use .loc to select by row label
# returns the row as a series whose index is the dataframe column names
df.loc[0]

In [ ]:
# use .loc to select single value by row label, column name
df.loc[0, "pct_below_poverty"]

In [ ]:
# slice of rows from label 5 to label 7, inclusive
# this returns a pandas dataframe
df.loc[5:7]

In [ ]:
# slice of rows from label 1 to label 3, inclusive
# slice of columns from pct_hispanic to pct_asian, inclusive
df.loc[1:3, "pct_hispanic":"pct_asian"]

In [ ]:
# subset of rows from with labels in list
# subset of columns with names in list
df.loc[[1, 3], ["pct_hispanic", "pct_asian"]]

In [ ]:
# you can use a column of unique identifiers as the index
# fips codes uniquely identify each row (but verify!)
df = df.set_index("GEOID10")
df.index.is_unique

In [ ]:
df.head()

In [ ]:
# .loc works by label, not by position in the dataframe
df.loc[0]

In [ ]:
# the index now contains fips codes, so you have to use .loc accordingly to select by row label
df.loc["06037137201"]

### 4c. Select by (integer) position

In [ ]:
# get the row in the zero-th position in the dataframe
df.iloc[0]

In [ ]:
# you can slice as well
# note, while .loc[] is inclusive, .iloc[] is not
# get the rows from position 0 up to but not including position 3 (ie, rows 0, 1, and 2)
df.iloc[0:3]

In [ ]:
# get the value from the row in position 3 and the column in position 2 (zero-indexed)
df.iloc[3, 2]

### 4d. Select/filter by value

You can subset or filter a dataframe for based on the values in its rows/columns.

In [ ]:
# filter the dataframe by rows with 30%+ rent burden
df[df["rent_burden"] > 0.3]

In [ ]:
# what exactly did that do? let's break it out.
df["rent_burden"] > 0.3

In [ ]:
# essentially a true/false mask that filters by value
mask = df["rent_burden"] > 0.3
df[mask]

In [ ]:
# you can chain multiple conditions together
# pandas logical operators are: | for or, & for and, ~ for not
# these must be grouped by using parentheses due to order of operations
# question: which tracts are both rent-burdened and majority-Black?
mask = (df["rent_burden"] > 0.3) & (df["pct_black"] > 50)
df[mask].shape

In [ ]:
# which tracts are both rent-burdened and either majority-Black or majority-Hispanic?
mask1 = df["rent_burden"] > 0.3
mask2 = df["pct_black"] > 50
mask3 = df["pct_hispanic"] > 50
mask = mask1 & (mask2 | mask3)
df[mask].shape

In [ ]:
# see the mask
mask

In [ ]:
# ~ means not... it essentially flips trues to falses and vice-versa
~mask

In [ ]:
# which rows are in a state that begins with "Cal"?
# all of them... because we're looking only at LA county
mask = df["state_name"].str.startswith("Cal")
df[mask].shape

In [ ]:
# now it's your turn
# create a new subset dataframe containing all the rows with median home values
# above $800,000 and percent-White above 60%... how many rows did you get?

## 5. Descriptive stats

In [ ]:
# what share of majority-White tracts are rent burdened?
mask1 = df["pct_white"] > 50
mask2 = mask1 & (df["rent_burden"] > 0.3)
len(df[mask2]) / len(df[mask1])

In [ ]:
# what share of majority-Hispanic tracts are rent burdened?
mask1 = df["pct_hispanic"] > 50
mask2 = mask1 & (df["rent_burden"] > 0.3)
len(df[mask2]) / len(df[mask1])

In [ ]:
# you can sort the dataframe by values in some column
df.sort_values("pct_below_poverty", ascending=False).dropna().head()

In [ ]:
# use the describe() method to pull basic descriptive stats for some column
df["med_household_income"].describe()

#### Or if you need the value of a single stat, call it directly

Key measures of central tendency: mean and median

In [ ]:
# the mean, or "average" value
df["med_household_income"].mean()

In [ ]:
# the median, or "typical" (ie, 50th percentile) value
df["med_household_income"].median()

In [ ]:
# now it's your turn
# create a new subset dataframe containing rows with median household income above the (tract)
# average in LA county... what is the median median home value across this subset of tracts?

Key measures of dispersion or variability: range, IQR, variance, standard deviation

In [ ]:
df["med_household_income"].min()

In [ ]:
# which tract has the lowest median household income?
df["med_household_income"].idxmin()

In [ ]:
df["med_household_income"].max()

In [ ]:
# what is the 90th-percentile value?
df["med_household_income"].quantile(0.90)

In [ ]:
# calculate the distribution's range
df["med_household_income"].max() - df["med_household_income"].min()

In [ ]:
# calculate its IQR
df["med_household_income"].quantile(0.75) - df["med_household_income"].quantile(0.25)

In [ ]:
# calculate its variance... rarely used in practice
df["med_household_income"].var()

In [ ]:
# calculate its standard deviation
# this is the sqrt of the variance... putting it into same units as the variable itself
df["med_household_income"].std()

In [ ]:
# now it's your turn
# what's the average (mean) median home value across majority-White tracts?
# And across majority-Black tracts?

## 6. Merge and concatenate

### 6a. Merging DataFrames

In [ ]:
# create a subset dataframe with only race/ethnicity variables
race_cols = ["pct_asian", "pct_black", "pct_hispanic", "pct_white"]
df_race = df[race_cols]
df_race.head()

In [ ]:
# create a subset dataframe with only economic variables
econ_cols = ["med_home_value", "med_household_income"]
df_econ = df[econ_cols].sort_values("med_household_income")
df_econ.head()

In [ ]:
# merge them together, aligning rows based on their labels in the index
df_merged = pd.merge(left=df_econ, right=df_race, how="inner", left_index=True, right_index=True)
df_merged.head()

In [ ]:
# reset df_econ's index
df_econ = df_econ.reset_index()
df_econ.head()

In [ ]:
# merge them together, aligning rows based on their labels in the index
# doesn't work! their indexes do not share any labels to match/align the rows
df_merged = pd.merge(left=df_econ, right=df_race, how="inner", left_index=True, right_index=True)
df_merged

In [ ]:
# now it's your turn
# change the "how" argument: what happens if you try an "outer" join?
# or a "left" join? or a "right" join?

In [ ]:
# instead merge where df_race index matches df_econ GEOID10 column
df_merged = pd.merge(left=df_econ, right=df_race, how="inner", left_on="GEOID10", right_index=True)
df_merged.head()

### 6b. Concatenating DataFrames

In [ ]:
# load the orange county tracts data
filepath = "https://raw.githubusercontent.com/gboeing/ppd534/main/data/census_tracts_data_oc.csv"
oc = pd.read_csv(filepath, dtype={"GEOID10": str})
oc = oc.set_index("GEOID10")
oc.shape

In [ ]:
oc.head()

In [ ]:
# merging joins data together aligned by the index, but concatenating just smushes
# it together along some axis
df_all = pd.concat([df, oc], sort=False)
df_all

## 7. Grouping and summarizing

In [ ]:
# extract county fips from index then replace with friendly name
df_all["county"] = df_all.index.str.slice(2, 5)
df_all["county"] = df_all["county"].replace({"037": "LA", "059": "OC"})
df_all["county"]

In [ ]:
# group the rows by county
counties = df_all.groupby("county")

In [ ]:
# what is the median pct_white across the tracts in each county?
counties["pct_white"].median()

In [ ]:
# look at several columns' medians by county
counties[["pct_bachelors_degree", "pct_foreign_born", "pct_commute_drive_alone"]].median()

In [ ]:
# now it's your turn
# group the tracts by county and find the highest/lowest tract percentages that speak English-only